#### 體積測試

In [8]:
from OCC.Display.SimpleGui import init_display
from OCC.Extend.DataExchange import read_iges_file

shapes = read_iges_file(r"C:\alan\ITRI\model\cube.IGS")
# 初始化 3D 顯示環境
display, start_display, add_menu, add_function_to_menu = init_display()
display.DisplayShape(shapes, update=True)
start_display()

In [2]:
import OCC.Extend.DataExchange
from OCC.Core.IGESControl import IGESControl_Reader
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop_VolumeProperties
from OCC.Display.SimpleGui import init_display

def read_iges_file(file_path):
    reader = IGESControl_Reader()
    status = reader.ReadFile(file_path)
    if status == 0:
        raise ValueError("Failed to read IGES file")
    reader.TransferRoots()
    return reader

def calculate_volume(iges_file):
    # Read the IGES file
    iges_reader = read_iges_file(iges_file)

    # Get the top level shape
    shape = iges_reader.OneShape()

    # Calculate the volume of the shape
    volume = 0.0
    if not shape.IsNull():
        mass_properties = GProp_GProps()
        brepgprop_VolumeProperties(shape, mass_properties)
        volume = mass_properties.Mass()
    
    return volume

# Example usage:
iges_file = r"C:\alan\ITRI\model\cube.IGS"
model_volume = calculate_volume(iges_file)
print("Model Volume:", model_volume)

Model Volume: -666666.6666666666


C:\Users\Martin\AppData\Local\Temp\ipykernel_5536\1813956207.py:26: DeprecationWarning: Call to deprecated function brepgprop_VolumeProperties since pythonocc-core 7.7.1. This function will be removed in a future release, please rather use the static method brepgprop.VolumeProperties
  brepgprop_VolumeProperties(shape, mass_properties)


#### 掃掠物體（填充物）

In [3]:
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeSphere, BRepPrimAPI_MakeCylinder
from OCC.Core.GC import GC_MakeSegment
from OCC.Core.GeomAPI import GeomAPI_PointsToBSpline
from OCC.Core.GeomLProp import GeomLProp_SurfaceTool
from OCC.Core.ShapeAnalysis import ShapeAnalysis_FreeBoundsProperties
from OCC.Core.BRepOffsetAPI import BRepOffsetAPI_ThruSections
from OCC.Core.GeomFill import GeomFill_Pipe
from OCC.Core.TopoDS import TopoDS_Compound
from OCC.Core.BRep import BRep_Tool_Surface
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeFace
from OCC.Core.TopExp import TopExp_Explorer

In [4]:
# 函數作用: 通過兩點生成線段
from OCC.Core.gp import gp_Pnt
from OCC.Core.GC import GC_MakeSegment
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire
from OCC.Display.SimpleGui import init_display
# 輸入: 兩個點
# 輸出: 線段
# def generate_segment(point1, point2):
#     aSegment = GC_MakeSegment(gp_Pnt(*point1), gp_Pnt(*point2))
#     anEdge = BRepBuilderAPI_MakeEdge(aSegment.Value())
#     aWire = BRepBuilderAPI_MakeWire(anEdge.Edge())
#     return aWire.Shape()

aSegment = GC_MakeSegment(gp_Pnt(0,0,0), gp_Pnt(15,0,0))
anEdge = BRepBuilderAPI_MakeEdge(aSegment.Value())
aWire = BRepBuilderAPI_MakeWire(anEdge.Edge())

#生成兩段線段
# E21=BRepBuilderAPI_MakeEdge(gp_Pnt(40,0,10),gp_Pnt(82.5,25,10)).Edge()
# E22=BRepBuilderAPI_MakeEdge(gp_Pnt(82.5,25,10),gp_Pnt(42.5,93,10)).Edge()
# W2=BRepBuilderAPI_MakeWire(E21,E22)

if __name__ == "__main__":
    display, start_display, add_menu, add_function_to_menu = init_display()
    display.DisplayShape(aWire.Shape(), update=True)
    # display.FitAll()
    start_display()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Martin\anaconda3\envs\pyoccenv\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "c:\Users\Martin\anaconda3\envs\pyoccenv\lib\site-packages\OCC\Display\tkDisplay.py", line 62, in Zoom
    self._display.ZoomFactor(zoom_factor)
UnboundLocalError: local variable 'zoom_factor' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Martin\anaconda3\envs\pyoccenv\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "c:\Users\Martin\anaconda3\envs\pyoccenv\lib\site-packages\OCC\Display\tkDisplay.py", line 62, in Zoom
    self._display.ZoomFactor(zoom_factor)
UnboundLocalError: local variable 'zoom_factor' referenced before assignment
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Martin\anaconda3\envs\pyoccenv\lib\tkinter\__init__.py", line 1892, in __call__


In [5]:
#根據引導軌跡掃出圖形
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire,BRepBuilderAPI_MakeFace
from OCC.Core.BRepOffsetAPI import BRepOffsetAPI_MakePipe
from OCC.Core.gp import gp_Circ, gp_Ax2, gp_Pnt, gp_Dir
from OCC.Display.SimpleGui import init_display

E21=BRepBuilderAPI_MakeEdge(gp_Pnt(40,0,10),gp_Pnt(82.5,25,10)).Edge()
E22=BRepBuilderAPI_MakeEdge(gp_Pnt(82.5,25,10),gp_Pnt(42.5,93,10)).Edge()
W2=BRepBuilderAPI_MakeWire(E21,E22)

#(以圓作為截面)
c=gp_Circ(gp_Ax2(gp_Pnt(0,0,0),gp_Dir(42.5,25,0)),10)

Ec = BRepBuilderAPI_MakeEdge(c).Edge()
Wc = BRepBuilderAPI_MakeWire(Ec).Wire()
F=BRepBuilderAPI_MakeFace(Wc, True)
S=BRepOffsetAPI_MakePipe(W2.Wire(), F.Shape())#第一个参数轨迹线,第二个参数是轮廟


display, start_display, add_menu, add_function_to_menu = init_display()
display.DisplayShape(S.Shape(),update=True)
start_display()

In [15]:
#根據引導軌跡掃出圖形
from OCC.Core.gp import gp_Pnt, gp_Vec, gp_Dir
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire
import numpy as np

#input
#normal_vector
normal_vector=np.array((0,1,0))
magnitude = np.linalg.norm(normal_vector)# 將向量除以其範數以得到單位向量
normal_vector = normal_vector/ magnitude
print(normal_vector)

#two point_position
point_start=np.array((0,0,0))
point_end=np.array((15,0,0))
tan_vector=point_start-point_end
print(tan_vector)

#direction_1
dir_1= np.cross(normal_vector, tan_vector)
magnitude = np.linalg.norm(dir_1)# 將向量除以其範數以得到單位向量
dir_1 = dir_1/ magnitude

# 定義截面為方形
# 定義方形的四個頂點
point_1=point_start+normal_vector*2+dir_1*2
point_2=point_start+normal_vector*2-dir_1*2
point_3=point_start-normal_vector*5-dir_1*5
point_4=point_start-normal_vector*5+dir_1*5
p1 = gp_Pnt(point_1[0], point_1[1], point_1[2])
p2 = gp_Pnt(point_2[0], point_2[1], point_2[2])
p3 = gp_Pnt(point_3[0], point_3[1], point_3[2])
p4 = gp_Pnt(point_4[0], point_4[1], point_4[2])
print(point_1)
print(point_2)
print(point_3)
print(point_4)
# 使用這些頂點創建方形的邊緣
edge1 = BRepBuilderAPI_MakeEdge(p1, p2).Edge()
edge2 = BRepBuilderAPI_MakeEdge(p2, p3).Edge()
edge3 = BRepBuilderAPI_MakeEdge(p3, p4).Edge()
edge4 = BRepBuilderAPI_MakeEdge(p4, p1).Edge()

# 使用這些邊緣創建方形的線框
square_wire = BRepBuilderAPI_MakeWire(edge1, edge2, edge3, edge4).Wire()

# 將線框轉換為面，作為截面
F = BRepBuilderAPI_MakeFace(square_wire, True)
S=BRepOffsetAPI_MakePipe(aWire.Wire(), F.Shape())#第一个参数轨迹线,第二个参数是轮廟


display, start_display, add_menu, add_function_to_menu = init_display()
display.DisplayShape(S.Shape(),update=True)
start_display()

[0. 1. 0.]
[-15   0   0]
[0. 2. 2.]
[ 0.  2. -2.]
[ 0. -5. -5.]
[ 0. -5.  5.]


#### 相減測試

In [10]:
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeSphere
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Cut
from OCC.Display.SimpleGui import init_display

# 创建一个长方体
box = BRepPrimAPI_MakeBox(10, 10, 10).Shape()

# 创建一个球体
sphere = BRepPrimAPI_MakeSphere(gp_Pnt(5, 5, 5), 6).Shape()

# 创建一个几何切割对象
cut_algo = BRepAlgoAPI_Cut(box, sphere)

# 执行几何切割操作
cut_result = cut_algo.Shape()

# 显示结果
display, start_display, add_menu, add_function_to_menu = init_display()
# display.DisplayShape(box, update=True)
# display.DisplayShape(sphere, update=True)
display.DisplayShape(cut_result, update=True)
start_display()

In [58]:
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeSphere
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Common
from OCC.Display.SimpleGui import init_display

# 创建一个长方体
box = BRepPrimAPI_MakeBox(10, 10, 10).Shape()

# 创建一个球体
sphere = BRepPrimAPI_MakeSphere(gp_Pnt(5, 5, 5), 6).Shape()

# 创建一个几何相交对象
common_algo = BRepAlgoAPI_Common(box, sphere)

# 执行几何相交操作
common_result = common_algo.Shape()

# 显示结果
display, start_display, add_menu, add_function_to_menu = init_display()
# display.DisplayShape(box, update=True)
# display.DisplayShape(sphere, update=True)
display.DisplayShape(common_result, update=True)
start_display()

In [16]:
from OCC.Core.gp import gp_Pnt
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox, BRepPrimAPI_MakeSphere
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Cut
from OCC.Display.SimpleGui import init_display


# 创建一个球体
sphere = BRepPrimAPI_MakeSphere(gp_Pnt(5, 0, 0), 6).Shape()

# 创建一个几何切割对象
cut_algo = BRepAlgoAPI_Cut(sphere,S.Shape() )

# 执行几何切割操作
cut_result = cut_algo.Shape()

# 显示结果
display, start_display, add_menu, add_function_to_menu = init_display()
# display.DisplayShape(box, update=True)
# display.DisplayShape(sphere, update=True)
display.DisplayShape(cut_result, update=True)
start_display()